In [1]:
import pandas as pd

df = pd.read_parquet("Hackaton2025/StockUniteLegale_utf8.parquet")
df_hist = pd.read_parquet("Hackaton2025/StockUniteLegaleHistorique_utf8.parquet")

df = df[["siren", "dateCreationUniteLegale", "trancheEffectifsUniteLegale", "categorieEntreprise", "dateDebut", "etatAdministratifUniteLegale",
    "nomUniteLegale", "activitePrincipaleUniteLegale", "categorieJuridiqueUniteLegale", "nomenclatureActivitePrincipaleUniteLegale", "nicSiegeUniteLegale" ]]

df_hist = df_hist[["siren","dateDebut","etatAdministratifUniteLegale"]]

df_final = pd.concat([df,df_hist])

# Suppression des lignes strinctement identique
df_final = df_final.drop_duplicates()


df_final.to_parquet("Hackaton2025/StockUniteLegale_Clean.parquet",index=False)

In [34]:
df = pd.read_parquet("Hackaton2025/StockUniteLegale_Clean.parquet")

In [3]:
df.shape

(98210861, 11)

In [5]:
df["siren"].nunique()

28760238

In [6]:
# Visualisation d'une ligne
for i in df.columns:
    print(i, df[i].iloc[0])

siren 000325175
dateCreationUniteLegale 2000-09-26
trancheEffectifsUniteLegale NN
categorieEntreprise PME
dateDebut 2018-02-07
etatAdministratifUniteLegale A
nomUniteLegale JANOYER
activitePrincipaleUniteLegale 32.12Z
categorieJuridiqueUniteLegale 1000.0
nomenclatureActivitePrincipaleUniteLegale NAFRev2
nicSiegeUniteLegale 65.0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98210861 entries, 0 to 98210860
Data columns (total 11 columns):
 #   Column                                     Dtype  
---  ------                                     -----  
 0   siren                                      object 
 1   dateCreationUniteLegale                    object 
 2   trancheEffectifsUniteLegale                object 
 3   categorieEntreprise                        object 
 4   dateDebut                                  object 
 5   etatAdministratifUniteLegale               object 
 6   nomUniteLegale                             object 
 7   activitePrincipaleUniteLegale              object 
 8   categorieJuridiqueUniteLegale              float64
 9   nomenclatureActivitePrincipaleUniteLegale  object 
 10  nicSiegeUniteLegale                        float64
dtypes: float64(2), object(9)
memory usage: 8.0+ GB


In [8]:
# Visualisation des nan
for i in df.columns:
    print(i, f" : {((df[i].isna().sum()) / len(df) )*100} %") 

siren  : 0.0 %
dateCreationUniteLegale  : 71.90950907150687 %
trancheEffectifsUniteLegale  : 70.71582744804569 %
categorieEntreprise  : 89.26170090291745 %
dateDebut  : 0.8302890247546043 %
etatAdministratifUniteLegale  : 2.2478929290722744 %
nomUniteLegale  : 83.46302350409086 %
activitePrincipaleUniteLegale  : 70.73774559414564 %
categorieJuridiqueUniteLegale  : 70.71582744804569 %
nomenclatureActivitePrincipaleUniteLegale  : 70.73774559414564 %
nicSiegeUniteLegale  : 70.71582744804569 %


In [9]:
# Suppression des nan non problèmatique
df = df[df["etatAdministratifUniteLegale"].isna() == False]
df = df[df["dateDebut"].isna() == False]

In [10]:
df.shape

(95187752, 11)

In [11]:
df["siren"].nunique()

28352521

In [37]:
# 1️⃣ Convertir dateDebut en datetime
df["dateDebut"] = pd.to_datetime(df["dateDebut"], errors="coerce")

In [39]:
df["dateDebut"].max()

Timestamp('2232-01-01 00:00:00')

In [13]:
# 1️⃣ Convertir dateDebut en datetime
df["dateDebut"] = pd.to_datetime(df["dateDebut"], errors="coerce")

# 2️⃣ Extraire uniquement l’année
df["annee"] = df["dateDebut"].dt.year

# 3️⃣ Supprimer la colonne d’origine (optionnel)
df = df.drop(columns=["dateDebut"])

# 4️⃣ Supprimer les doublons pour ne garder qu’une ligne par siren et par annee
df = df.drop_duplicates(subset=["siren", "annee"])

In [ ]:
# selection des années et suppression des années abérrantes.
df[(df["dateDebut"] > "2020-01-01") | (df["dateDebut"] < "2025-01-01")]

In [17]:
df.describe()

,categorieJuridiqueUniteLegale,nicSiegeUniteLegale,annee
count,2.835252e+07,2.835252e+07,5.772328e+07
mean,3.246854e+03,1.938498e+01,2.009416e+03
std,2.739824e+03,4.416437e+01,1.322517e+01
min,1.000000e+03,1.000000e+00,1.819000e+03
25%,1.000000e+03,1.300000e+01,2.004000e+03
50%,1.000000e+03,1.600000e+01,2.011000e+03
75%,5.710000e+03,2.100000e+01,2.019000e+03
max,9.970000e+03,7.666500e+04,2.232000e+03


In [16]:
df.to_csv("dataset_clean.csv",index=False)